In [1]:
import torch
import gc


# Step 2: Call torch.cuda.empty_cache() to free up GPU memory
torch.cuda.empty_cache()

# Step 3: Call garbage collector to ensure all unreferenced memory is freed
gc.collect()

# Optionally, you can also reset the default CUDA stream
torch.cuda.synchronize()

# Check if GPU memory is cleared
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())


0
0


In [2]:
import torch

# Check PyTorch GPU
print("Is CUDA available for PyTorch? ", torch.cuda.is_available())
print("Number of GPUs available: ", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print("GPU ", i, ": ", torch.cuda.get_device_name(i))
import torch
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)



Is CUDA available for PyTorch?  True
Number of GPUs available:  1
GPU  0 :  NVIDIA GeForce RTX 4070
True
True


In [3]:
import cudf

def load_specific_chunk(csv_file_path, start_row, chunk_size):
    # Adjust `skiprows` handling here:
    # If `start_row` is greater than 0, calculate `skiprows` as before.
    # Otherwise, explicitly set `skiprows` to 0 instead of `None`.
    skiprows = 1 + start_row if start_row > 0 else 0
    
    # Now, `skiprows` is guaranteed to be an integer, which should avoid the TypeError.
    df_chunk = cudf.read_csv(csv_file_path, skiprows=skiprows, nrows=chunk_size, header=None if skiprows else 'infer')
    
    return df_chunk

# Usage example
csv_file_path = 'images_dataset.csv'
chunk_size = 100
# Load chunks sequentially
first_chunk = load_specific_chunk(csv_file_path, 0, chunk_size)
df=first_chunk

In [4]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(3, 16, kernel_size=2),
            nn.BatchNorm2d(16),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),

            nn.ConvTranspose2d(16, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),
            
            nn.ConvTranspose2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),

            nn.ConvTranspose2d(64, 128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),

            nn.ConvTranspose2d(128, 256, kernel_size=3),
            nn.BatchNorm2d(256),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),

            nn.ConvTranspose2d(256, 3, kernel_size=3),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 256, kernel_size=4, padding=1),
            nn.BatchNorm2d(256),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),

            nn.Conv2d(256, 128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),

            nn.Conv2d(128, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),

            nn.Conv2d(64, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),
            
            nn.Conv2d(32, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.SELU(inplace=True),
            nn.Dropout(0.25),

            nn.Conv2d(16, 1, kernel_size=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Reshape the input if it's flattened
        if x.dim() == 2:
            x = x.view(x.size(0), 3, 256, 256)
        x = self.main(x)
        return x.view(x.size(0), -1).mean(dim=1, keepdim=True)


# Instantiation and summary display
generator = Generator()
discriminator = Discriminator()

# Assuming you're using GPU for training
device = torch.device("cuda")
generator.to(device)
discriminator.to(device)

print(generator)
print(discriminator)


Generator(
  (main): Sequential(
    (0): ConvTranspose2d(3, 16, kernel_size=(2, 2), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SELU(inplace=True)
    (3): Dropout(p=0.25, inplace=False)
    (4): ConvTranspose2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): SELU(inplace=True)
    (7): Dropout(p=0.25, inplace=False)
    (8): ConvTranspose2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): SELU(inplace=True)
    (11): Dropout(p=0.25, inplace=False)
    (12): ConvTranspose2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): SELU(inplace=True)
    (15): Dropout(p=0.25, inplace=False)
    (16): ConvTranspose2d(128, 256, kernel_size=(3, 3), stri

In [5]:
df.columns

Index(['datetime', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3', 'pixel_4',
       'pixel_5', 'pixel_6', 'pixel_7', 'pixel_8',
       ...
       'pixel_196598', 'pixel_196599', 'pixel_196600', 'pixel_196601',
       'pixel_196602', 'pixel_196603', 'pixel_196604', 'pixel_196605',
       'pixel_196606', 'pixel_196607'],
      dtype='object', length=196609)

In [6]:
import pandas as pd
dt = df.pop('datetime')
# pd.set_option('display.max_rows', None)

In [7]:
dt

0                 1
1                10
2                11
3                12
4                13
          ...      
95    20230726_0000
96    20230726_0015
97    20230726_0030
98    20230726_0045
99    20230726_0100
Name: datetime, Length: 100, dtype: object

In [8]:
dt=dt.to_numpy()

In [9]:
import pandas as pd
import numpy as np

# Ensure 'dt' is a pandas Series
dt = pd.Series(dt)

# Convert properly formatted entries
mask = dt.str.match(r'\d{8}_\d{4}')
dt.loc[mask] = pd.to_datetime(dt[mask], format='%Y%m%d_%H%M')

# Convert numeric entries to float for interpolation
dt.loc[~mask] = pd.to_numeric(dt[~mask], errors='coerce')

# Interpolate numeric values
dt = dt.interpolate()

# Generate a date range for the entire series
start_date = dt.loc[mask].min()
full_range = pd.date_range(start=start_date, periods=len(dt), freq='15T')

# Replace interpolated numeric values with proper datetimes
dt.loc[~mask] = full_range[~mask]

print("After converting and interpolating:")
print(dt)

After converting and interpolating:
0     2023-07-25 08:00:00
1     2023-07-25 08:15:00
2     2023-07-25 08:30:00
3     2023-07-25 08:45:00
4     2023-07-25 09:00:00
             ...         
95    2023-07-26 00:00:00
96    2023-07-26 00:15:00
97    2023-07-26 00:30:00
98    2023-07-26 00:45:00
99    2023-07-26 01:00:00
Length: 100, dtype: object


/tmp/ipykernel_5681/486844610.py:15: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  dt = dt.interpolate()
/tmp/ipykernel_5681/486844610.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=start_date, periods=len(dt), freq='15T')


In [10]:
import cudf
import pandas as pd

dt_df = pd.DataFrame(dt, columns=['datetime'])
time = cudf.DataFrame.from_pandas(dt_df)
df = cudf.concat([df, time], axis=1)

/tmp/ipykernel_5681/892543047.py:4: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The DataFrame constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  dt_df = pd.DataFrame(dt, columns=['datetime'])


In [11]:
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

device = torch.device("cuda")

# Move models to the selected device
generator.to(device)
discriminator.to(device)

# BCE Loss function
criterion = nn.BCELoss()

# Optimizers
optimizerG = optim.AdamW(generator.parameters(), lr=0.0003)
optimizerD = optim.AdamW(discriminator.parameters(), lr=0.0003)

# After defining your optimizers
scheduler_G = StepLR(optimizerG, step_size=30, gamma=0.1)
scheduler_D = StepLR(optimizerD, step_size=30, gamma=0.1)

# Learning rate scheduler
schedulerG = optim.lr_scheduler.ReduceLROnPlateau(optimizerG, patience=5, factor=0.1)
schedulerD = optim.lr_scheduler.ReduceLROnPlateau(optimizerD, patience=5, factor=0.1)

num_epochs = 1  # For testing purposes
patience = 5
best_loss = float('inf')
patience_counter = 0


In [12]:
import torch
from torch.utils.data import DataLoader
import cudf
import cupy as cp

# Assuming 'df' is your cuDF DataFrame
# Calculate total number of rows
total_rows = len(df)

# Calculate split indices
train_end = int(total_rows * 0.8)
validate_end = train_end + int(total_rows * 0.15)

# Split the DataFrame into train, validate, and test sets
train_df = df.iloc[:train_end]
validate_df = df.iloc[train_end:validate_end]
test_df = df.iloc[validate_end:]

# Define batch size
batch_size = 16



In [13]:
train_df = train_df.to_pandas()
validate_df = validate_df.to_pandas()
test_df = test_df.to_pandas()

In [14]:
import os
os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class ImageDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe.reset_index(drop=True)  # Reset index to ensure it starts from 0
        self.pixel_columns = [col for col in dataframe.columns if col.startswith('pixel_')]
        self.datetime_column = 'datetime'

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        pixel_data = self.dataframe.iloc[idx][self.pixel_columns].values.astype('float32')
        datetime_data = pd.to_datetime(self.dataframe.iloc[idx][self.datetime_column]).timestamp()
        
        pixel_tensor = torch.tensor(pixel_data, dtype=torch.float32)
        datetime_tensor = torch.tensor(datetime_data, dtype=torch.float32)
        
        return pixel_tensor, datetime_tensor

# Assuming train_df, validate_df, and test_df are your DataFrames
train_dataset = ImageDataset(train_df)
validate_dataset = ImageDataset(validate_df)
test_dataset = ImageDataset(test_df)


batch_size = 8
num_workers = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)



In [16]:
batch_counter = 0
num_batches_to_print = 4

for pixel_batch, datetime_batch in train_loader:
    print(f"Batch {batch_counter + 1}:")
    print("Pixel data:")
    print(pixel_batch)
    print(f"Pixel data size: {pixel_batch.size()}")
    print("\nDatetime data:")
    print(datetime_batch)
    print(f"Datetime data size: {datetime_batch.size()}")
    print("\n" + "="*50 + "\n")
    
    batch_counter += 1
    
    if batch_counter >= num_batches_to_print:
        break

Batch 1:
Pixel data:
tensor([[0.2431, 0.2980, 0.2314,  ..., 0.7412, 0.7608, 0.7373],
        [0.1333, 0.1333, 0.1333,  ..., 0.6471, 0.6471, 0.6471],
        [0.1647, 0.1647, 0.1647,  ..., 0.6275, 0.6275, 0.6275],
        ...,
        [0.1569, 0.1569, 0.1569,  ..., 0.4784, 0.4784, 0.4784],
        [0.1216, 0.1216, 0.1216,  ..., 0.3843, 0.3843, 0.3843],
        [0.1843, 0.1843, 0.1843,  ..., 0.3647, 0.3647, 0.3647]])
Pixel data size: torch.Size([8, 196608])

Datetime data:
tensor([1.6903e+09, 1.6903e+09, 1.6903e+09, 1.6903e+09, 1.6903e+09, 1.6903e+09,
        1.6903e+09, 1.6903e+09])
Datetime data size: torch.Size([8])


Batch 2:
Pixel data:
tensor([[0.2941, 0.2941, 0.2941,  ..., 0.5412, 0.5412, 0.5412],
        [0.2157, 0.2157, 0.2157,  ..., 0.7686, 0.7686, 0.7686],
        [0.1922, 0.2471, 0.1804,  ..., 0.7294, 0.7686, 0.7216],
        ...,
        [0.1451, 0.2000, 0.1333,  ..., 0.7137, 0.7333, 0.7098],
        [0.1333, 0.1725, 0.1294,  ..., 0.7216, 0.7412, 0.7176],
        [0.0980, 0.

In [17]:
for epoch in range(num_epochs):
    generator.train()
    discriminator.train()
    for i, (real_data, datetime_data) in enumerate(train_loader):
        batch_size = real_data.size(0)
        real_data = real_data.to(device)
        datetime_data = datetime_data.unsqueeze(1).float().to(device)

        # Train Discriminator
        discriminator.zero_grad()
        
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        output = discriminator(real_data)
        d_loss_real = criterion(output, real_labels)
        
        noise = torch.randn(batch_size, 3, 256, 256, device=device)
        fake_data = generator(noise)
        output = discriminator(fake_data.detach())
        d_loss_fake = criterion(output, fake_labels)
        
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizerD.step()
        scheduler_D.step()

        # Train Generator
        generator.zero_grad()
        output = discriminator(fake_data)
        g_loss = criterion(output, real_labels)
        g_loss.backward()
        optimizerG.step()
        scheduler_G.step()
        
        if i % 100 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Step [{i}/{len(train_loader)}], '
                  f'd_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

    # Validation phase
    generator.eval()
    discriminator.eval()
    
    val_d_loss = 0.0
    val_g_loss = 0.0
    with torch.no_grad():
        for real_data, datetime_data in validate_loader:
            real_data = real_data.to(device)
            datetime_data = datetime_data.unsqueeze(1).float().to(device)
            batch_size = real_data.size(0)
            
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            
            output = discriminator(real_data)
            d_loss_real = criterion(output, real_labels)
            
            noise = torch.randn(batch_size, 3, 256, 256, device=device)
            fake_data = generator(noise)
            output = discriminator(fake_data)
            d_loss_fake = criterion(output, fake_labels)
            
            val_d_loss += (d_loss_real + d_loss_fake).item()
            
            output = discriminator(fake_data)
            g_loss = criterion(output, real_labels)
            
            val_g_loss += g_loss.item()
    
    val_d_loss /= len(validate_loader)
    val_g_loss /= len(validate_loader)
    
    schedulerG.step(val_g_loss)
    schedulerD.step(val_d_loss)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation D Loss: {val_d_loss:.4f}, Validation G Loss: {val_g_loss:.4f}")

    # Early stopping and model saving
    if val_g_loss < best_loss:
        best_loss = val_g_loss
        patience_counter = 0
        torch.save(generator.state_dict(), 'best_generator.pth')
        torch.save(discriminator.state_dict(), 'best_discriminator.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

# Test phase
generator.load_state_dict(torch.load('best_generator.pth'))
discriminator.load_state_dict(torch.load('best_discriminator.pth'))

generator.eval()
discriminator.eval()

test_d_loss = 0.0
test_g_loss = 0.0
with torch.no_grad():
    for real_data, datetime_data in test_loader:
        real_data = real_data.to(device)
        datetime_data = datetime_data.unsqueeze(1).float().to(device)
        batch_size = real_data.size(0)
        
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        output = discriminator(real_data)
        d_loss_real = criterion(output, real_labels)
        
        noise = torch.randn(batch_size, 3, 256, 256, device=device)
        fake_data = generator(noise)
        output = discriminator(fake_data)
        d_loss_fake = criterion(output, fake_labels)
        
        test_d_loss += (d_loss_real + d_loss_fake).item()
        
        output = discriminator(fake_data)
        g_loss = criterion(output, real_labels)
        
        test_g_loss += g_loss.item()

test_d_loss /= len(test_loader)
test_g_loss /= len(test_loader)

print(f"Test D Loss: {test_d_loss:.4f}, Test G Loss: {test_g_loss:.4f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 510.00 MiB. GPU 

Generalising everything  